## 📘 Однонаправленный LSTM от посимвольной до пословной токенизации

<details> 
    <summary><em><strong> Долгая краткосрочная память (LSTM)</strong></em></summary>

## 1. Введение и мотивация

### 1.1 История создания LSTM: ключевые работы Хохрайтера и Шмидхубера, развитие идеи

**Зарождение идеи (1991-1997)**

Проблема затухающего градиента была формально идентифицирована в начале 1990-х годов Сеппом Хохрайтером в его диссертационной работе. Хохрайтер и Юрген Шмидхубер начали искать архитектуру, которая могла бы преодолеть эту проблему.

**Ключевые этапы:**

- **1991-1995**: Первые эксперименты и теоретические разработки. Хохрайтер и Шмидхубер исследовали различные способы позволить градиентам течь через длинные последовательности без затухания.

- **1997**: Публикация оригинальной статьи "Long Short-Term Memory" в журнале Neural Computation. В этой фундаментальной работе была представлена архитектура LSTM с механизмом вентилей (gates) и постоянным потоком ошибки (Constant Error Carousel, CEC).

**Эволюция LSTM:**

- **1999-2000**: Феликс Герс и его коллеги представили "peephole connections" — модификацию, которая позволяет вентилям "заглядывать" в ячейку памяти.

- **2000**: Герс и Шмидхубер вводят "forget gate" (вентиль забывания) — критическое улучшение, позволяющее LSTM сбрасывать своё состояние и обучаться на последовательностях неограниченной длины.

- **2005**: Грейвс и Шмидхубер представили двунаправленный LSTM (BiLSTM), который обрабатывает последовательность в обоих направлениях.

- **2013-2014**: Эпоха широкого применения LSTM в промышленности, особенно в области распознавания речи и машинного перевода.

- **2014**: Разработка GRU (Gated Recurrent Unit) командой Чо как более простой альтернативы LSTM, сохраняющей большинство преимуществ.

**Цитата Шмидхубера о создании LSTM (2015):**

> "Проблема заключалась в том, что градиенты либо затухали, либо взрывались, и нам нужно было создать архитектуру, которая позволила бы информации и градиентам течь через много временных шагов."

### 1.2 Ключевые преимущества: почему LSTM стали стандартом в индустрии

**1. Решение проблемы затухающего градиента**

LSTM эффективно решает проблему затухающего градиента благодаря своему уникальному механизму ячейки памяти с контролируемыми вентилями. Ключевой компонент — **константный поток ошибки** (Constant Error Carousel, CEC), который обеспечивает неизменный градиент через состояние ячейки.

**2. Долговременная память**

LSTM способны запоминать информацию на сотни и даже тысячи временных шагов:
- Демонстрируют превосходную способность улавливать зависимости на больших расстояниях (long-range dependencies)
- Могут избирательно сохранять важную информацию и забывать неважную
- Позволяют моделировать контекст на различных временных масштабах одновременно

**3. Адаптивность к различным типам данных**

LSTM эффективно работают с разнообразными типами последовательных данных:
- Текст и речь (машинный перевод, распознавание речи)
- Временные ряды (финансовые прогнозы, данные сенсоров)
- Биологические последовательности (анализ ДНК, белков)
- Мультимодальные данные (подписи к изображениям, видеоаналитика)

**4. Масштабируемость и гибкость**

- Возможность создания глубоких архитектур путем штабелирования LSTM слоев
- Комбинируемость с другими типами нейронных сетей (CNN, Attention)
- Эффективная параллелизация обучения на современном оборудовании

**5. Промышленные результаты**

До появления трансформеров (2017-2018), LSTM были абсолютным стандартом в индустрии и научных исследованиях:

- **Google** (2015-2016): использовал LSTM в системах распознавания речи, сократив ошибки на 30%
- **Apple**: внедрил LSTM в Siri для улучшения понимания контекста
- **Facebook**: применял LSTM для автоматического перевода сообщений
- **Amazon**: использовал LSTM в рекомендательных системах и прогнозировании спроса

Даже после появления трансформеров, LSTM остаются востребованными в ряде областей:
- Обработка потоковых данных в реальном времени
- Задачи с ограниченными вычислительными ресурсами
- Приложения, требующие интерпретируемости модели

**Эволюция популярности LSTM** показывает их значимость: от академических исследований в конце 1990-х до промышленного доминирования в середине 2010-х, и последующую интеграцию с архитектурами на основе внимания (attention).

## 2. Архитектура LSTM: Как это работает?

### 2.1 Интуиция: Метафора конвейера с контролируемыми воротами

Чтобы понять принцип работы LSTM, представим его как умную производственную линию с системой конвейеров и ворот:

**1. Основной конвейер — ячейка памяти (cell state)**

Ключевой компонент LSTM – это **состояние ячейки (cell state)** – горизонтальная линия, проходящая через всю цепочку. Представьте длинный конвейер (или конвейерную ленту), который тянется через всю фабрику (последовательность). На этом конвейере перемещается "контейнер с информацией" (cell state, $C_t$).

![Image_01.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/LSTM/Image_01.png)

**Особенности состояния ячейки:**  
- Она проходит напрямую через всю цепочку, участвуя лишь в нескольких линейных преобразованиях.  
- Информация может легко течь по ней, не подвергаясь изменениям.  
- Этот контейнер может сохранять информацию практически неизменной на протяжении долгого времени — в этом главный секрет LSTM.  

Тем не менее, LSTM может **удалять информацию** из состояния ячейки; этот процесс регулируется структурами, называемыми **фильтрами (gates)**.  

**2. Система управляемых ворот (gates)**

На каждом шаге (временной точке) наш конвейер проходит через три контрольных пункта:

- **Вентиль забывания (Forget Gate)**: действует как фильтр, решающий, какую информацию удалить из контейнера. Представьте рабочего, который смотрит на содержимое контейнера и текущий вход, а затем решает, что выбросить. "Нам всё ещё нужно помнить пол субъекта, чтобы правильно согласовывать местоимения? Да, сохраняем. А информация о цвете его машины? Нет, выбрасываем."

- **Вентиль входа (Input Gate)**: определяет, какую новую информацию добавить в контейнер. Представьте другого рабочего, который, посмотрев на текущий вход и предыдущий выход, решает, какие новые факты достаточно важны, чтобы их запомнить. "Мы только что узнали имя нового персонажа в рассказе? Это важно, добавляем в контейнер."

- **Вентиль выхода (Output Gate)**: контролирует, какую часть содержимого контейнера передать внешнему миру (в виде скрытого состояния). Представьте третьего рабочего, который решает, какие части накопленной информации наиболее актуальны прямо сейчас. "Нам нужно предсказать следующее слово — важно знать, что подлежащее в единственном числе, но не важно знать, о какой стране шла речь ранее."

**3. Двойная система состояний**

В отличие от ванильной RNN, у LSTM есть две линии передачи информации:

- **Ячейка памяти (Cell State)** $C_t$: основной конвейер, предназначенный для долговременного хранения информации.
- **Скрытое состояние (Hidden State)** $h_t$: фильтрованная версия ячейки памяти, содержащая только ту информацию, которую сеть считает актуальной в текущий момент.

**Метафора с записной книжкой:**

Другой способ представить LSTM — это человек с записной книжкой (cell state), который постоянно решает:
- Какие старые заметки стереть (forget gate)
- Какие новые заметки записать (input gate)
- Какую информацию из книжки использовать при ответе на вопрос (output gate)

Эта система позволяет LSTM действовать как умный накопитель информации, избирательно запоминающий важные факты и игнорирующий неважные детали — именно то, что нужно для обработки длинных последовательностей.

### 2.2 Формализация и обозначения: определение размерностей и переменных

Давайте формализуем архитектуру LSTM, четко определив все компоненты и их размерности. Это поможет как в понимании структуры, так и при последующей реализации.

**Основные обозначения:**

| **Символ** | **Размерность** | **Описание** |
|------------|-----------------|--------------|
| $x_t$ | $\mathbb{R}^{d_x}$ | Входной вектор на шаге $t$ |
| $h_t$ | $\mathbb{R}^{d_h}$ | Скрытое состояние на шаге $t$ |
| $C_t$ | $\mathbb{R}^{d_h}$ | Состояние ячейки на шаге $t$ |
| $f_t$ | $\mathbb{R}^{d_h}$ | Активация вентиля забывания на шаге $t$ |
| $i_t$ | $\mathbb{R}^{d_h}$ | Активация вентиля входа на шаге $t$ |
| $o_t$ | $\mathbb{R}^{d_h}$ | Активация вентиля выхода на шаге $t$ |
| $\tilde{C}_t$ | $\mathbb{R}^{d_h}$ | Кандидат-вектор новых значений ячейки |

**Весовые матрицы и векторы смещения:**

| **Символ** | **Размерность** | **Описание** |
|------------|-----------------|--------------|
| $W_f$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для вентиля забывания |
| $W_i$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для вентиля входа |
| $W_C$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для кандидат-вектора |
| $W_o$ | $\mathbb{R}^{d_h \times (d_x + d_h)}$ | Веса для вентиля выхода |
| $b_f$ | $\mathbb{R}^{d_h}$ | Смещение для вентиля забывания |
| $b_i$ | $\mathbb{R}^{d_h}$ | Смещение для вентиля входа |
| $b_C$ | $\mathbb{R}^{d_h}$ | Смещение для кандидат-вектора |
| $b_o$ | $\mathbb{R}^{d_h}$ | Смещение для вентиля выхода |

**Функции активации:**
- $\sigma$: сигмоидальная функция, отображает входы в диапазон [0, 1]
- $\tanh$: гиперболический тангенс, отображает входы в диапазон [-1, 1]

**Размерности входных и выходных данных:**
- $d_x$: размерность входного вектора $x_t$
- $d_h$: размерность скрытого состояния и состояния ячейки

**Конкатенация входа и предыдущего состояния:**

Для упрощения записи мы часто используем конкатенацию входного вектора $x_t$ и предыдущего скрытого состояния $h_{t-1}$:

$$[x_t, h_{t-1}] \in \mathbb{R}^{d_x + d_h}$$

Это позволяет нам определить веса как одну матрицу для каждого вентиля, вместо разделения на отдельные матрицы для $x_t$ и $h_{t-1}$.

**Примечания по размерностям:**

1. В стандартной архитектуре LSTM размерность состояния ячейки $C_t$ равна размерности скрытого состояния $h_t$. В некоторых вариациях они могут различаться.

2. Все вентили ($f_t$, $i_t$, $o_t$) имеют одинаковую размерность $d_h$, что позволяет им поэлементно контролировать состояние ячейки.

3. Общее количество параметров в стандартном LSTM:
   - Веса: $4 \times d_h \times (d_x + d_h)$
   - Смещения: $4 \times d_h$
   - Итого: $4 \times d_h \times (d_x + d_h + 1)$

Эти обозначения мы будем использовать в следующих разделах для описания математических формул и динамики LSTM.

### 2.3 Динамика одного шага

Теперь рассмотрим, как точно вычисляются все компоненты LSTM на одном временном шаге $t$. Разберем каждый элемент архитектуры подробно.

### Вентиль забывания (forget gate)

Вентиль забывания $f_t$ определяет, какую информацию из предыдущего состояния ячейки $C_{t-1}$ следует сохранить, а какую — стереть:

$$
f_t = \sigma\big(W_f \cdot [x_t, h_{t-1}] + b_f\big)
$$

Здесь:
- $[x_t, h_{t-1}]$ — конкатенация текущего входа и предыдущего скрытого состояния
- $W_f$ — весовая матрица для вентиля забывания
- $b_f$ — вектор смещения
- $\sigma$ — сигмоидальная функция, возвращающая значения в интервале [0, 1]

Результат $f_t$ представляет собой вектор значений между 0 и 1, где:
- **1** означает "полностью сохранить эту информацию"
- **0** означает "полностью забыть эту информацию"

### **Пример:** 

Если модель работает с текстом и в какой-то момент понимает, что начинается новое предложение, вентиль забывания может "обнулить" часть информации о предыдущем предложении.

![Image_02.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/LSTM/Image_02.png)

**Контекст:** Обработка последовательности "Я люблю море. На небе светит солнце."

1. **Состояние после первого предложения:**
   - $C_{t-1}$ (состояние ячейки) кодирует информацию о первом предложении: [0.8, 0.6, -0.2]
     - 0.8 → факт "море" (существительное)
     - 0.6 → эмоция "любовь"
     - -0.2 → местоимение "я"

2. **Обработка слова "На" (начало нового предложения):**
   - Вход $x_t$ = embedding слова "На" [0.1, -0.3, 0.5]
   - $h_{t-1}$ = предыдущее скрытое состояние [0.7, 0.5, -0.1]
   - Вентиль забывания вычисляет:
     $$
     f_t = \sigma\left(
     \begin{bmatrix}
     0.2 & 0.4 & -0.1 \\
     -0.3 & 0.6 & 0.2 \\
     0.1 & -0.2 & 0.3
     \end{bmatrix}
     \cdot
     \begin{bmatrix}
     0.1 \\ -0.3 \\ 0.5 \\ 0.7 \\ 0.5 \\ -0.1
     \end{bmatrix}
     +
     \begin{bmatrix}
     0.1 \\ -0.2 \\ 0.3
     \end{bmatrix}
     \right) = [0.1, 0.9, 0.8]
     $$
     - Первый нейрон (0.1) → забыть информацию о местоимении (уже не нужно)
     - Второй нейрон (0.9) → сохранить эмоциональный контекст
     - Третий нейрон (0.8) → сохранить информацию о существительном

3. **Обновленное состояние ячейки:**
   - $C_t = f_t \odot C_{t-1} = [0.1, 0.9, 0.8] \odot [0.8, 0.6, -0.2] = [0.08, 0.54, -0.16]$
     - Значение 0.8 (море) уменьшилось до 0.08 → забыто
     - Эмоция 0.6 сохранилась как 0.54
     - Местоимение -0.2 стало -0.016 → почти забыто

**Интерпретация:** Модель решила:
- Сохранить эмоциональный контекст (может пригодиться для анализа тональности)
- Забыть конкретные существительные из предыдущего предложения
- Подготовиться к новой синтаксической структуре (новое предложение)

**Визуализация векторов:**
```
До forget gate:    [ 0.80  0.60  -0.20 ]
After forget gate: [ 0.08  0.54  -0.02 ]
                   │      │      └── Почти забыто ("я")
                   │      └────────── Сохранено ("любовь")
                   └───────────────── Забыто ("море")
```

---

### Вентиль входа (input gate)

Вентиль входа $i_t$ решает, какую новую информацию добавить в состояние ячейки:

$$
i_t = \sigma\big(W_i \cdot [x_t, h_{t-1}] + b_i\big)
$$

Как и в случае с вентилем забывания, результатом является вектор значений в интервале [0, 1], где:
- **1** означает "полностью добавить эту новую информацию"
- **0** означает "не добавлять эту информацию"

#### Кандидат-вектор состояния ячейки

![Image_03.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/LSTM/Image_03.png)

Параллельно с вентилем входа, создается кандидат-вектор $\tilde{C}_t$ — "черновик" новой информации, которую потенциально можно добавить в состояние ячейки:

$$
\tilde{C}_t = \tanh\big(W_C \cdot [x_t, h_{t-1}] + b_C\big)
$$

Здесь используется функция $\tanh$, чтобы значения были нормализованы в диапазоне [-1, 1].

#### Обновление состояния ячейки

![Image_04.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/LSTM/Image_04.png)

Теперь мы готовы обновить состояние ячейки $C_t$, используя вентиль забывания, вентиль входа и кандидат-вектор:

$$
C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t
$$

где $\odot$ обозначает поэлементное умножение (умножение Адамара).

Это уравнение описывает ключевой механизм LSTM:
1. $f_t \odot C_{t-1}$ — старая информация, которую мы решили сохранить
2. $i_t \odot \tilde{C}_t$ — новая информация, которую мы решили добавить

**Важно!** Состояние ячейки $C_t$ обновляется с помощью только линейных операций (умножение и сложение). Это гарантирует, что градиент может течь через ячейку без затухания, решая проблему ванильных RNN.

### **Пример:**

Продолжим обработку последовательности "Я люблю море. На небе светит солнце." после применения вентиля забывания.

1. **Текущее состояние после вентиля забывания:**
   - $f_t \odot C_{t-1} = [0.08, 0.54, -0.16]$ — часть информации о предыдущем предложении сохранена

2. **Обработка слова "На" (начало нового предложения):**
   - Вход $x_t$ = embedding слова "На" [0.1, -0.3, 0.5]
   - $h_{t-1}$ = предыдущее скрытое состояние [0.7, 0.5, -0.1]
   
   - Вентиль входа вычисляет:
     $$
     i_t = \sigma\left(
     \begin{bmatrix}
     0.3 & -0.2 & 0.1 \\
     0.5 & 0.4 & -0.3 \\
     -0.1 & 0.7 & 0.2
     \end{bmatrix}
     \cdot
     \begin{bmatrix}
     0.1 \\ -0.3 \\ 0.5 \\ 0.7 \\ 0.5 \\ -0.1
     \end{bmatrix}
     +
     \begin{bmatrix}
     -0.1 \\ 0.2 \\ 0.1
     \end{bmatrix}
     \right) = [0.7, 0.6, 0.9]
     $$
     
   - Кандидат-вектор нового состояния:
     $$
     \tilde{C}_t = \tanh\left(
     \begin{bmatrix}
     0.4 & 0.1 & -0.3 \\
     -0.2 & 0.5 & 0.3 \\
     0.3 & -0.4 & 0.2
     \end{bmatrix}
     \cdot
     \begin{bmatrix}
     0.1 \\ -0.3 \\ 0.5 \\ 0.7 \\ 0.5 \\ -0.1
     \end{bmatrix}
     +
     \begin{bmatrix}
     0.2 \\ -0.1 \\ 0.4
     \end{bmatrix}
     \right) = [0.6, -0.3, 0.7]
     $$
     - Первый нейрон (0.6) → информация о месте "небо" (существительное)
     - Второй нейрон (-0.3) → нейтральная эмоциональная тональность
     - Третий нейрон (0.7) → предлог "на" (указывает на местоположение)

3. **Обновленное состояние ячейки с новой информацией:**
   - $i_t \odot \tilde{C}_t = [0.7, 0.6, 0.9] \odot [0.6, -0.3, 0.7] = [0.42, -0.18, 0.63]$
   
   - Итоговое состояние ячейки:
     $C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t = [0.08, 0.54, -0.16] + [0.42, -0.18, 0.63] = [0.50, 0.36, 0.47]$
     - Значение 0.08 (забытое "море") + 0.42 (новое "небо") = 0.50 → новая информация о месте
     - Эмоция 0.54 (сохраненная "любовь") + (-0.18) (нейтральность) = 0.36 → снижение эмоциональной окраски
     - Значение -0.16 (почти забытое "я") + 0.63 (новое "на") = 0.47 → переход от субъекта к локации

**Интерпретация:** Модель:
- Добавила новую информацию о небе, которое становится новым существительным
- Снизила эмоциональную окраску, переходя к более нейтральному описанию
- Переключила фокус с субъекта ("я") на пространственное отношение ("на")

**Визуализация векторов:**
```
После forget gate:  [ 0.08  0.54  -0.16 ]
Новая информация:   [ 0.42  -0.18  0.63 ]
Итоговое состояние: [ 0.50  0.36  0.47 ]
                     │      │      └── Новый фокус (пространство "на")
                     │      └───────── Снижение эмоциональности
                     └──────────────── Новое существительное ("небо")
```
---

#### Вентиль выхода (output gate)

![Image_05.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/LSTM/Image_05.png)

Вентиль выхода $o_t$ определяет, какую часть обновленного состояния ячейки передать в выходное скрытое состояние:

$$
o_t = \sigma\big(W_o \cdot [x_t, h_{t-1}] + b_o\big)
$$

Как и другие вентили, $o_t$ содержит значения в интервале [0, 1].

#### Скрытое состояние

Наконец, вычисляем новое скрытое состояние $h_t$, применяя вентиль выхода к нормализованному состоянию ячейки:

$$
h_t = o_t \odot \tanh(C_t)
$$

Здесь:
- $\tanh(C_t)$ нормализует значения состояния ячейки до диапазона [-1, 1]
- $o_t$ определяет, какие компоненты этого нормализованного состояния передать дальше

Скрытое состояние $h_t$ используется как для предсказания выхода на текущем шаге, так и как вход для следующего шага сети.

**Итог: полный набор формул LSTM**

Для удобства, вот полный набор формул, описывающих один шаг LSTM:

$$
\begin{align}
f_t &= \sigma(W_f \cdot [x_t, h_{t-1}] + b_f) \\
i_t &= \sigma(W_i \cdot [x_t, h_{t-1}] + b_i) \\
\tilde{C}_t &= \tanh(W_C \cdot [x_t, h_{t-1}] + b_C) \\
C_t &= f_t \odot C_{t-1} + i_t \odot \tilde{C}_t \\
o_t &= \sigma(W_o \cdot [x_t, h_{t-1}] + b_o) \\
h_t &= o_t \odot \tanh(C_t)
\end{align}
$$

Эти шесть уравнений полностью описывают динамику LSTM ячейки на одном временном шаге.

## 3. Математические основы и функционирование LSTM

### 3.1 Роль сигмоидальных функций: Почему именно сигмоида для вентилей

Сигмоидальная функция играет ключевую роль в архитектуре LSTM, особенно в механизме вентилей. Разберем, почему именно эта функция активации используется для всех трех вентилей (забывания, входа и выхода).

![Image_06.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_&_18/assets/LSTM/Image_06.jpg)

**Математическое определение сигмоидальной функции:**

$\sigma(x) = \frac{1}{1 + e^{-x}}$

**Ключевые свойства сигмоиды, делающие её идеальной для вентилей:**

1. **Ограниченный диапазон выходных значений [0, 1]**
   - Это свойство критически важно для вентилей, так как они должны выполнять функцию "фильтра"
   - Значение 0 означает "полностью блокировать информацию"
   - Значение 1 означает "полностью пропустить информацию"
   - Промежуточные значения позволяют частично пропускать информацию

2. **Гладкость и дифференцируемость**
   - Сигмоида непрерывна и дифференцируема на всей области определения
   - Её производная имеет простую форму: $\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))$
   - Это свойство важно для обратного распространения градиента при обучении

3. **Нелинейность и насыщение**
   - При больших положительных значениях $x$ функция стремится к 1
   - При больших отрицательных значениях $x$ функция стремится к 0
   - Это создает эффект "насыщения", который стабилизирует динамику сети

**Практическое применение в вентилях LSTM:**

- **Вентиль забывания ($f_t$)**: сигмоида определяет, какой процент каждого элемента в состоянии ячейки сохранить. Значение 0 означает "забыть полностью", 1 — "сохранить полностью".

- **Вентиль входа ($i_t$)**: сигмоида контролирует, какую часть новой информации ($\tilde{C}_t$) добавить в состояние ячейки. Значение 0 означает "не добавлять ничего", 1 — "добавить полностью".

- **Вентиль выхода ($o_t$)**: сигмоида регулирует, какую часть информации из состояния ячейки передать в скрытое состояние $h_t$. Значение 0 означает "ничего не передавать", 1 — "передать всё".

**Примечание по инициализации смещений:**

Важно отметить, что смещения (bias) вентилей часто инициализируются специальным образом:
- Смещение вентиля забывания ($b_f$) часто инициализируется положительными значениями (например, 1 или 2), чтобы в начале обучения сеть была склонна "помнить" информацию
- Смещения других вентилей обычно инициализируются нулями или небольшими случайными значениями

Такая инициализация помогает LSTM быстрее обучаться работе с долговременными зависимостями.

### 3.2 Функция активации tanh: Её роль в кандидат-векторе и выходе

Гиперболический тангенс (tanh) — вторая ключевая функция активации в архитектуре LSTM. Она используется в двух важных местах: при создании кандидат-вектора и при формировании выходного скрытого состояния.

![Image_07.png](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-17_%26_18/assets/LSTM/Image_07.JPG)

**Математическое определение функции tanh:**

$\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$

**Ключевые свойства tanh, важные для LSTM:**

1. **Ограниченный диапазон выходных значений [-1, 1]**
   - В отличие от сигмоиды, tanh симметрична относительно начала координат
   - Диапазон [-1, 1] позволяет представлять как положительные, так и отрицательные активации с равной амплитудой

2. **Крутизна градиента**
   - Производная tanh в нуле равна 1, что больше, чем у сигмоиды (0.25)
   - Это обеспечивает более сильный градиент при обратном распространении

3. **Нулевое среднее значение**
   - Выходы функции tanh имеют приблизительно нулевое среднее значение
   - Это помогает бороться с проблемой смещения при обучении (covariate shift)

**Роль tanh в кандидат-векторе $\tilde{C}_t$:**

$\tilde{C}_t = \tanh(W_C \cdot [x_t, h_{t-1}] + b_C)$

1. **Нормализация значений**
   - tanh приводит все значения к диапазону [-1, 1], что создает стабильную динамику в ячейке
   - Это предотвращает неконтролируемый рост значений в состоянии ячейки

2. **Биполярность представления**
   - Отрицательные значения могут представлять "ингибирующие" сигналы
   - Положительные значения могут представлять "возбуждающие" сигналы
   - Это важно для создания богатого внутреннего представления данных

3. **Баланс с сигмоидой вентиля входа**
   - tanh создает кандидат-значения в диапазоне [-1, 1]
   - Сигмоида вентиля входа определяет, какую часть этих значений добавить
   - Это позволяет как добавлять, так и вычитать информацию из состояния ячейки

**Роль tanh при формировании скрытого состояния $h_t$:**

$h_t = o_t \odot \tanh(C_t)$

1. **Нормализация выходных значений**
   - Состояние ячейки $C_t$ может содержать значения с большой амплитудой
   - tanh нормализует эти значения перед выходом, что важно для стабильности последующих слоев

2. **Балансировка активаций**
   - tanh обеспечивает симметричный выход для положительных и отрицательных значений в $C_t$
   - Это полезно для последующих слоев, которые часто лучше работают с центрированными входами

3. **Интерпретируемость выхода**
   - Скрытое состояние $h_t$ используется для предсказаний и как входной сигнал для следующего шага
   - Нормализованный диапазон [-1, 1] обеспечивает согласованное масштабирование этих сигналов

**Сравнение с другими функциями активации:**

Почему именно tanh, а не другие функции активации, такие как ReLU?

- **ReLU** не ограничивает верхний предел выходных значений, что может привести к неконтролируемому росту активаций
- **Leaky ReLU** имеет неограниченный диапазон и асимметричный отклик, что менее подходит для состояния ячейки
- **Sigmoid** ограничивает выход только положительными значениями, что уменьшает выразительность модели

**Практический аспект:**

В некоторых современных вариациях LSTM функция tanh может заменяться на другие активации для кандидат-вектора, но в классической архитектуре и большинстве практических реализаций tanh остается стандартным выбором, благодаря её математическим свойствам, которые хорошо согласуются с природой задачи обработки последовательностей.

### 3.3 Поток градиентов в LSTM: как архитектура решает проблему затухающего градиента

Ключевое преимущество LSTM перед обычными RNN — способность эффективно обрабатывать длинные последовательности без проблемы затухающего градиента. Рассмотрим, как именно LSTM решает эту фундаментальную проблему на уровне потока градиентов.

**Напомним проблему в ванильных RNN:**

В обычной RNN градиент потери по скрытому состоянию $h_{t-k}$ включает произведение множества якобианов:

$$\frac{\partial h_t}{\partial h_{t-k}} = \prod_{i=t-k+1}^{t} \frac{\partial h_i}{\partial h_{i-1}} = \prod_{i=t-k+1}^{t} \text{diag}(\tanh'(a_i)) \cdot W_{hh}$$

Эти якобианы обычно имеют собственные значения меньше 1, что приводит к экспоненциальному затуханию градиента при увеличении $k$.

**Ключевая инновация LSTM: Константный поток ошибки**

Главная особенность LSTM — **Константный поток ошибки (Constant Error Carousel, CEC)**, который обеспечивается прямым линейным соединением через состояние ячейки $C_t$.

Рассмотрим поток градиента через состояние ячейки от момента $t$ к моменту $t-1$:

$$\frac{\partial C_t}{\partial C_{t-1}} = \frac{\partial (f_t \odot C_{t-1} + i_t \odot \tilde{C}_t)}{\partial C_{t-1}} = f_t$$

Это выражение показывает критическое свойство LSTM: **градиент от $C_t$ к $C_{t-1}$ проходит через простое поэлементное умножение на вентиль забывания $f_t$**. Не используются ни нелинейные функции активации, ни матричные умножения — только прямое поэлементное умножение.

**Рекуррентное распространение градиента через несколько шагов:**

При распространении градиента через $k$ шагов назад имеем:

$$\frac{\partial C_t}{\partial C_{t-k}} = \prod_{i=t-k+1}^{t} \frac{\partial C_i}{\partial C_{i-1}} = \prod_{i=t-k+1}^{t} f_i$$

Это произведение векторов вентилей забывания (применяемое поэлементно).

**Как это решает проблему затухающего градиента:**

1. **Контроль потока градиентов через $f_t$**
   - Если компоненты $f_t$ близки к 1, градиент протекает почти без затухания
   - LSTM обучается устанавливать $f_t \approx 1$ для важной информации

2. **Аддитивное обновление состояния ячейки**
   - В отличие от мультипликативных рекуррентных соединений в ванильных RNN, LSTM использует аддитивное обновление:
     $C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$
   - Это позволяет градиентам течь без обязательного умножения на вес рекуррентной связи

3. **Механизм обучаемого "забывания"**
   - Вместо фиксированного затухания, LSTM обучается тому, что необходимо помнить, а что можно забыть
   - Это как избирательный "шлюз", который пропускает важные градиенты и блокирует неважные

**Математическое моделирование потока градиентов:**

Полный градиент потери $L$ по состоянию ячейки $C_{t-k}$ можно разложить:

$$\frac{\partial L}{\partial C_{t-k}} = \sum_{j=t-k+1}^{T} \frac{\partial L}{\partial C_j} \frac{\partial C_j}{\partial C_{t-k}}$$

Здесь первый член $\frac{\partial L}{\partial C_j}$ — это обратное распространение от потери к состоянию ячейки в момент $j$, а второй член $\frac{\partial C_j}{\partial C_{t-k}}$ — это произведение вентилей забывания по пути от $t-k$ до $j$.

**Практические следствия:**

1. **Длинные зависимости**
   - LSTM может обучаться зависимостям на сотни и даже тысячи шагов, что невозможно для ванильных RNN
   - Например, LSTM может связать "Франция" в начале текста с "французский" в конце, даже если между ними большой промежуток

2. **Выборочная чувствительность**
   - LSTM обучается быть чувствительной только к важным долговременным зависимостям
   - Это более эффективно, чем попытка запоминать всё подряд

3. **Стабильность обучения**
   - Контролируемый поток градиентов делает обучение LSTM более стабильным
   - Реже требуется gradient clipping или специальная инициализация весов

Таким образом, уникальная архитектура LSTM с константным потоком ошибки через состояние ячейки и обучаемыми вентилями эффективно решает проблему затухающего градиента, что позволяет моделировать долговременные зависимости в последовательных данных.

### 3.4 Сравнение с ванильной RNN: Математический взгляд на преимущества

Давайте проведем строгое математическое сравнение LSTM и ванильной RNN, чтобы лучше понять, почему LSTM справляется с задачей обработки последовательностей значительно лучше.

**1. Архитектурное сравнение основных уравнений**

**Ванильная RNN:**
$h_t = \tanh(W_{xh}x_t + W_{hh}h_{t-1} + b_h)$

**LSTM:**
$
\begin{align}
f_t &= \sigma(W_f \cdot [x_t, h_{t-1}] + b_f) \\
i_t &= \sigma(W_i \cdot [x_t, h_{t-1}] + b_i) \\
\tilde{C}_t &= \tanh(W_C \cdot [x_t, h_{t-1}] + b_C) \\
C_t &= f_t \odot C_{t-1} + i_t \odot \tilde{C}_t \\
o_t &= \sigma(W_o \cdot [x_t, h_{t-1}] + b_o) \\
h_t &= o_t \odot \tanh(C_t)
\end{align}
$

**Ключевые различия:**

- **Одно vs несколько уравнений**: RNN использует одно уравнение, в то время как LSTM разделяет обновление на несколько специализированных компонентов.
- **Одно vs два состояния**: RNN имеет только скрытое состояние $h_t$, тогда как LSTM разделяет информацию между скрытым состоянием $h_t$ и состоянием ячейки $C_t$.
- **Простое vs адаптивное обновление**: RNN всегда обновляет всё скрытое состояние целиком, а LSTM избирательно обновляет компоненты состояния ячейки.

**2. Поток информации во времени**

**Ванильная RNN:**

Информация от входа $x_{t-k}$ к текущему скрытому состоянию $h_t$ проходит через цепочку нелинейных преобразований:

$h_t = F(h_{t-1}, x_t) = F(F(h_{t-2}, x_{t-1}), x_t) = ... = F(F(...F(h_{t-k-1}, x_{t-k})...), x_t)$

Здесь $F(h, x) = \tanh(W_{xh}x + W_{hh}h + b_h)$. Каждое применение нелинейности $\tanh$ может приводить к потере информации.

**LSTM:**

Информация может течь через состояние ячейки $C_t$ с контролируемым забыванием:

$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t = f_t \odot (f_{t-1} \odot C_{t-2} + i_{t-1} \odot \tilde{C}_{t-1}) + i_t \odot \tilde{C}_t = ...$

Раскрывая это выражение дальше, получаем:

$C_t = \left( \prod_{j=t-k+1}^{t} f_j \right) \odot C_{t-k} + \sum_{j=t-k+1}^{t} \left( i_j \odot \tilde{C}_j \odot \prod_{l=j+1}^{t} f_l \right)$

Это показывает, что состояние ячейки $C_t$ является взвешенной суммой всех предыдущих входов, где веса определяются произведениями вентилей $f_j$. Если все $f_j \approx 1$, информация может протекать почти без искажений.

**3. Математический анализ потока градиентов**

**Ванильная RNN:**

Градиент потери $L$ по весам $W_{hh}$ вычисляется с помощью цепного правила:

$\frac{\partial L}{\partial W_{hh}} = \sum_{k=1}^{t} \frac{\partial L}{\partial h_t} \frac{\partial h_t}{\partial h_{t-k}} \frac{\partial h_{t-k}}{\partial W_{hh}}$

Где вторая производная содержит произведение якобианов:

$\frac{\partial h_t}{\partial h_{t-k}} = \prod_{j=t-k+1}^{t} \frac{\partial h_j}{\partial h_{j-1}} = \prod_{j=t-k+1}^{t} \text{diag}(\tanh'(W_{xh}x_j + W_{hh}h_{j-1} + b_h)) \cdot W_{hh}$

Собственные значения этого произведения обычно меньше 1, что приводит к затуханию градиента.

**LSTM:**

У LSTM градиент от $C_t$ к $C_{t-k}$ вычисляется как:

$\frac{\partial C_t}{\partial C_{t-k}} = \prod_{j=t-k+1}^{t} \frac{\partial C_j}{\partial C_{j-1}} = \prod_{j=t-k+1}^{t} f_j$

Поскольку $f_j$ — это результат сигмоидальной функции, обученной специально для контроля потока информации, LSTM может поддерживать значения $f_j \approx 1$ для важных компонентов, что предотвращает затухание градиента.

**4. Количественное сравнение параметров и вычислительной сложности**

**Ванильная RNN:**
- Количество параметров: $d_h \times (d_x + d_h + 1)$
- Вычислительная сложность на шаг: $O(d_h \times (d_x + d_h))$

**LSTM:**
- Количество параметров: $4 \times d_h \times (d_x + d_h + 1)$
- Вычислительная сложность на шаг: $O(4 \times d_h \times (d_x + d_h))$

LSTM требует примерно в 4 раза больше параметров и вычислений, но это компенсируется значительным улучшением производительности на задачах с долговременными зависимостями.

**5. Эмпирическое сравнение возможностей**

| **Свойство** | **Ванильная RNN** | **LSTM** |
|--------------|-------------------|----------|
| Максимальная длина зависимостей | 5-10 шагов | Сотни или тысячи шагов |
| Устойчивость к шуму | Низкая | Высокая |
| Способность забывать неважную информацию | Ограниченная | Высокая |
| Адаптивность к различным временным масштабам | Низкая | Высокая |

**6. Геометрическая интерпретация**

Если представить пространство скрытых состояний как многомерное пространство, то:

- **Ванильная RNN** создает сложную нелинейную динамику, в которой траектории могут быстро сходиться к аттракторам, что приводит к потере информации о прошлых состояниях.

- **LSTM** создает управляемую динамику, где важные направления в пространстве состояний могут сохраняться почти без изменений, позволяя информации течь на большие расстояния, в то время как неважные направления могут быстро затухать.

В целом, LSTM представляет собой глубоко продуманное расширение архитектуры RNN, которое целенаправленно устраняет ключевые математические ограничения ванильных RNN, особенно проблему затухающего градиента, что делает LSTM значительно более мощным инструментом для моделирования последовательностей с долговременными зависимостями.

</details>

### Подключим необходимые пакеты

In [ ]:
"""
Реализация моделей на основе LSTM с различной глубиной для генерации текста.

Данный модуль обеспечивает создание, обучение и применение однонаправленных LSTM-сетей
для генерации текста на русском языке. Реализованы два подхода: символьная и словесная токенизация.
Модели имеют настраиваемую глубину (количество LSTM-слоев) для анализа влияния 
сложности архитектуры на качество генерации.
"""

# Библиотеки для работы с данными и базами данных
import sqlite3
from collections import Counter
from typing import List, Dict, Any, Optional, Union, Tuple

import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Библиотеки для обработки текста
import nltk
from nltk.tokenize import sent_tokenize

# Библиотеки глубокого обучения
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Библиотеки для визуализации и анализа
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Утилиты
from tqdm import tqdm

# Настройка визуализации
seaborn.set(palette='summer')

# Определение устройства для вычислений
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Загрузим данные

In [3]:
conn = sqlite3.connect('/kaggle/input/wikibooks-dataset/wikibooks.sqlite')

df = pd.read_sql_query("SELECT * FROM ru LIMIT 3300", conn)

In [ ]:
# Извлечение предложений из текстов
sentences = []

for sentence in tqdm(df['body_text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='russian') if len(x) < 256]
    )
    
print("Количество предложений", len(sentences))

100%|██████████| 3300/3300 [00:10<00:00, 304.16it/s]

Количество предложений 120873


### Train loop

In [ ]:
def fit_epoch(
    model: nn.Module, 
    train_loader: DataLoader, 
    criterion: nn.Module, 
    optimizer: torch.optim.Optimizer, 
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> Tuple[float, float]:
    """
    Description:
    ---------------
        Выполняет одну эпоху обучения модели.

    Args:
    ---------------
        model: Модель для обучения
        train_loader: Загрузчик обучающих данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        sheduler: Планировщик скорости обучения (опционально)

    Returns:
    ---------------
        Tuple[float, float]: Перплексия и значение функции потерь
    """
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    losses = []
    perplexity = []
    
    for batch in train_loader:
        optimizer.zero_grad()

        # Распространение прямое и обратное
        logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
        loss = criterion(
            logits, batch['target_ids'].flatten()
        )
        loss.backward()
        optimizer.step()
        
        # Сохранение метрик
        perplexity.append(torch.exp(loss).item())
        losses.append(loss.item())
        
    # Расчет средних значений метрик
    perplexity_avg = sum(perplexity) / len(perplexity)
    losses_avg = sum(losses) / len(losses)    
    
    return perplexity_avg, losses_avg


def eval_epoch(
    model: nn.Module, 
    val_loader: DataLoader, 
    criterion: nn.Module
) -> Tuple[float, float]:
    """
    Description:
    ---------------
        Оценивает модель на валидационном наборе данных.

    Args:
    ---------------
        model: Модель для оценки
        val_loader: Загрузчик валидационных данных
        criterion: Функция потерь

    Returns:
    ---------------
        Tuple[float, float]: Перплексия и значение функции потерь
    """
    model.eval()
    perplexity = []
    losses = []
    
    with torch.no_grad():
        for batch in val_loader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(
                logits,
                batch['target_ids'].flatten()
            )
            perplexity.append(torch.exp(loss).item())
            losses.append(loss.item())

    # Расчет средних значений метрик
    perplexity_avg = sum(perplexity) / len(perplexity)
    losses_avg = sum(losses) / len(losses)
    
    return perplexity_avg, losses_avg


def train(
    train_dataloader: DataLoader, 
    eval_dataloader: DataLoader, 
    model: nn.Module, 
    epochs: int, 
    ignore_index: int = char2ind['<pad>'],
    optimizer: Optional[torch.optim.Optimizer] = None, 
    criterion: Optional[nn.Module] = None, 
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> Tuple[nn.Module, List[Tuple[float, float, float, float]]]:
    """
    Description:
    ---------------
        Обучает модель на заданном количестве эпох.

    Args:
    ---------------
        train_dataloader: Загрузчик обучающих данных
        eval_dataloader: Загрузчик валидационных данных
        model: Модель для обучения
        epochs: Количество эпох обучения
        ignore_index: Индекс токена, который не учитывается в функции потерь
        optimizer: Оптимизатор (по умолчанию Adam)
        criterion: Функция потерь (по умолчанию CrossEntropyLoss)
        sheduler: Планировщик скорости обучения (опционально)

    Returns:
    ---------------
        Tuple[nn.Module, List[Tuple[float, float, float, float]]]: 
            Обученная модель и история обучения
    """
    # Инициализация оптимизатора и функции потерь, если не указаны
    if optimizer is None:
      optimizer = torch.optim.Adam(model.parameters())

    if criterion is None:
      criterion = nn.CrossEntropyLoss(ignore_index=ignore_index)

    # Сохранение лучших весов модели
    best_model_wts = model.state_dict()
    best_perplexity = 10e10

    # История обучения
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_perplexirty {t_acc:0.4f} val_perplexirty {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            # Обучение на одной эпохе
            train_perplexirty, train_loss = fit_epoch(
                model, train_dataloader, criterion, optimizer
            )
            
            # Валидация модели
            val_perplexirty, val_loss = eval_epoch(
                model, eval_dataloader, criterion
            )
            
            # Сохранение метрик
            history.append((train_loss, train_perplexirty, val_loss, val_perplexirty))
            
            # Сохранение лучшей модели
            if val_perplexirty < best_perplexity:
                best_perplexity = val_perplexirty
                best_model_wts = model.state_dict()

            # Обновление прогресс-бара и вывод результатов
            pbar_outer.update(1)
            tqdm.write(log_template.format(
                ep=epoch+1, 
                t_loss=train_loss,
                v_loss=val_loss, 
                t_acc=train_perplexirty, 
                v_acc=val_perplexirty
            ))

    print('Best val perplexirty: {:4f}'.format(best_perplexity))
    
    # Загрузка лучших весов
    model.load_state_dict(best_model_wts)

    return model, history

### Посимвольная токенизация

In [ ]:
# Обработка и токенизация на уровне символов
stop_chars = [
    "\t", ",", ".", "!", "@", "'", '"', ";", "\n", "(", ")",
    "[", "]", "{", "}", "?", ":", "-", "_", "+", "=", "^", "*", 
    "&", "`", "~"
]

chars = Counter()

for sentence in tqdm(sentences):
    for char in sentence:
        if char in stop_chars:
            continue
        chars[char] += 1
        
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
counter_threshold = 500

for char, cnt in chars.items():
    if cnt > counter_threshold:
        vocab.add(char)
        
print("Размер словаря:", len(vocab))

100%|██████████| 120873/120873 [00:09<00:00, 12853.64it/s]

Размер словаря: 91


In [7]:
char2ind = {char: i for i, char in enumerate(vocab)}
ind2char = {i: char for char, i in char2ind.items()}

In [ ]:
class CharDataset:
    """
    Description:
    ---------------
        Датасет для работы с посимвольной токенизацией текста.

    Args:
    ---------------
        sentences: Список предложений для обработки

    Examples:
    ---------------
        >>> dataset = CharDataset(sentences)
        >>> sample = dataset[0]
        >>> len(sample)
        # Длина токенизированного предложения
    """
    def __init__(self, sentences: List[str]) -> None:
        self.data = sentences
        self.unk_id = char2ind['<unk>']
        self.bos_id = char2ind['<bos>']
        self.eos_id = char2ind['<eos>']
        self.pad_id = char2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Преобразует предложение в последовательность индексов символов.

        Args:
        ---------------
            idx: Индекс предложения в датасете

        Returns:
        ---------------
            List[int]: Токенизированное предложение как список индексов
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            char2ind.get(char, self.unk_id) for char in self.data[idx]
        ]
        tokenized_sentence += [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает размер датасета.

        Returns:
        ---------------
            int: Количество предложений в датасете
        """
        return len(self.data)
    

class WordDataset:
    """
    Description:
    ---------------
        Датасет для работы с пословной токенизацией текста.

    Args:
    ---------------
        sentences: Список предложений для обработки

    Examples:
    ---------------
        >>> dataset = WordDataset(sentences)
        >>> sample = dataset[0]
        >>> len(sample)
        # Длина токенизированного предложения
    """
    def __init__(self, sentences: List[str]) -> None:
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Преобразует предложение в последовательность индексов слов.

        Args:
        ---------------
            idx: Индекс предложения в датасете

        Returns:
        ---------------
            List[int]: Токенизированное предложение как список индексов
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) 
            for word in nltk.word_tokenize(self.data[idx])
        ]
        tokenized_sentence += [self.eos_id]
        
        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает размер датасета.

        Returns:
        ---------------
            int: Количество предложений в датасете
        """
        return len(self.data)
    
    
    
def collate_fn_with_padding(
    input_batch: List[List[int]], 
    pad_id: int = char2ind['<pad>']
) -> Dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для преобразования батча данных: добавляет паддинг и 
        создает тензоры ввода и целевых значений.

    Args:
    ---------------
        input_batch: Пакет токенизированных предложений
        pad_id: Идентификатор токена для заполнения (паддинга)

    Returns:
    ---------------
        Dict[str, torch.Tensor]: Словарь с тензорами входных и целевых id
    """
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch


def generate_sequence(
    model: nn.Module, 
    dict_2ind: Dict[str, int], 
    ind2dict: Dict[int, str], 
    starting_seq: str, 
    max_seq_len: int = 256
) -> str:
    """
    Description:
    ---------------
        Генерирует текстовую последовательность, начиная с заданной.

    Args:
    ---------------
        model: Обученная языковая модель
        dict_2ind: Словарь для преобразования символов/слов в индексы
        ind2dict: Словарь для преобразования индексов в символы/слова
        starting_seq: Начальная последовательность
        max_seq_len: Максимальная длина генерируемой последовательности

    Returns:
    ---------------
        str: Сгенерированная последовательность

    Examples:
    ---------------
        >>> result = generate_sequence(model, char2ind, ind2char, "привет")
        >>> print(result)
        'привет мир...'
    """
    device = 'cpu'
    model = model.to(device)
    
    # Преобразование начальной последовательности в индексы
    input_ids = [dict_2ind['<bos>']] + [
        dict_2ind.get(char, dict_2ind['<unk>']) for char in starting_seq
    ]
    input_ids = torch.LongTensor(input_ids).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            # Получение распределения вероятностей для следующего символа/слова
            logits = model(input_ids.unsqueeze(0))
            next_token_logits = logits[0, -1, :]
            next_token = next_token_logits.argmax()
            
            # Добавление предсказанного токена к входной последовательности
            input_ids = torch.cat([input_ids, next_token.unsqueeze(0)])

            # Завершение генерации при появлении токена конца последовательности
            if next_token.item() == dict_2ind['<eos>']:
                break

    # Преобразование индексов обратно в текст
    words = ' '.join([ind2dict[idx.item()] for idx in input_ids])

    return words

Разобьём датасет на train и eval, так же определим dataloader для train и eval

In [ ]:
# Разделение данных и создание датасетов
train_sentences, eval_sentences = train_test_split(
    sentences, test_size=0.2
)

train_dataset = CharDataset(train_sentences)
eval_dataset  = CharDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

Определим архитектуру на основе LSTM

In [ ]:
# Определение архитектуры LSTM-модели
class LanguageModel(nn.Module):
    """
    Description:
    ---------------
        Языковая модель на основе LSTM с настраиваемым количеством слоев.

    Args:
    ---------------
        vocab_size: Размер словаря
        hidden_dim: Размерность скрытого состояния
        num_layers: Количество LSTM-слоев

    Examples:
    ---------------
        >>> model = LanguageModel(vocab_size=1000, hidden_dim=256, num_layers=2)
        >>> input_tensor = torch.LongTensor([[1, 2, 3, 4]])
        >>> output = model(input_tensor)
        >>> output.shape
        torch.Size([1, 4, 1000])
    """
    def __init__(
        self, 
        vocab_size: int, 
        hidden_dim: int, 
        num_layers: int = 1
    ) -> None:
        super().__init__()
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        
        # Создание списка LSTM-слоев
        self.lstm_layers = nn.ModuleList()
        for _ in range(num_layers):
            self.lstm_layers.append(
                nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
            )
            
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(0.2)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход модели.

        Args:
        ---------------
            input_batch: Тензор с индексами входных токенов

        Returns:
        ---------------
            torch.Tensor: Предсказанные логиты для каждого токена
        """
        # Преобразование индексов в эмбеддинги
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        
        # Пропуск через первый LSTM-слой
        output, _ = self.lstm_layers[0](embeddings)
        
        # Пропуск через дополнительные LSTM-слои с резидуальными соединениями
        for i in range(1, self.num_layers):
            output1, _ = self.lstm_layers[i](output)
            output = output1 + output  # Резидуальное соединение
        
        # Дополнительные слои
        output = self.dropout(
            self.linear(self.non_lin(output))
        )  # [batch_size, seq_len, hidden_dim]
        
        # Проекция на размер словаря
        projection = self.projection(
            self.non_lin(output)
        )  # [batch_size, seq_len, vocab_size]

        return projection

In [ ]:
# Создание и обучение однослойной LSTM-модели
model = LanguageModel(
    hidden_dim=256, 
    vocab_size=len(vocab), 
    num_layers=1
).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

LanguageModel(
  (embedding): Embedding(91, 256)
  (lstm_layers): ModuleList(
    (0): LSTM(256, 256, batch_first=True)
  )
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=91, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)
Number of model parameters: 638,811


In [ ]:
best_model, history = train(
    train_dataloader, 
    eval_dataloader, 
    model, 
    10
)

epoch:  10%|█         | 1/10 [00:38<05:50, 38.95s/it]


Epoch 001 train_loss: 2.2684     val_loss 1.8093 train_perplexirty 11.2871 val_perplexirty 6.1087


epoch:  20%|██        | 2/10 [01:17<05:07, 38.45s/it]


Epoch 002 train_loss: 1.7334     val_loss 1.6459 train_perplexirty 5.6688 val_perplexirty 5.1875


epoch:  30%|███       | 3/10 [01:55<04:27, 38.28s/it]

...


Epoch 008 train_loss: 1.4633     val_loss 1.4515 train_perplexirty 4.3217 val_perplexirty 4.2709


epoch:  90%|█████████ | 9/10 [05:44<00:38, 38.16s/it]


Epoch 009 train_loss: 1.4493     val_loss 1.4402 train_perplexirty 4.2615 val_perplexirty 4.2231


epoch: 100%|██████████| 10/10 [06:22<00:00, 38.21s/it]


Epoch 010 train_loss: 1.4380     val_loss 1.4309 train_perplexirty 4.2135 val_perplexirty 4.1838
Best val perplexirty: 4.183791


In [ ]:
# Тестирование генерации текста
generate_sequence(
    model, 
    char2ind, 
    ind2char, 
    starting_seq='источник связан с '
)

'<bos>источник связан с помощью программы в соответствии с помощью программы в соответствии с помощью программы в соответствии с помощью программы в соответствии с помощью программы в соответствии с помощью программы в соответствии с помощью программы в соответствии с помощью про'

In [ ]:
# Очистка памяти для следующей модели
import gc
torch.cuda.empty_cache()
gc.collect()

186

### Теперь добавим несколько слоев LSTM

In [ ]:
# Создание и обучение многослойной LSTM-модели
model = LanguageModel(
    hidden_dim=256, 
    vocab_size=len(vocab), 
    num_layers=3
).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

LanguageModel(
  (embedding): Embedding(91, 256)
  (lstm_layers): ModuleList(
    (0-2): 3 x LSTM(256, 256, batch_first=True)
  )
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=91, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)
Number of model parameters: 1,691,483


In [ ]:
best_model, history = train(
    train_dataloader, 
    eval_dataloader, 
    model, 
    10
)

epoch:  10%|█         | 1/10 [01:23<12:30, 83.39s/it]


Epoch 001 train_loss: 2.1932     val_loss 1.7179 train_perplexirty 10.6381 val_perplexirty 5.5749


epoch:  20%|██        | 2/10 [02:46<11:07, 83.39s/it]


Epoch 002 train_loss: 1.6379     val_loss 1.5451 train_perplexirty 5.1536 val_perplexirty 4.6904


epoch:  30%|███       | 3/10 [04:10<09:43, 83.35s/it]

...


Epoch 008 train_loss: 1.3167     val_loss 1.3255 train_perplexirty 3.7324 val_perplexirty 3.7654


epoch:  90%|█████████ | 9/10 [12:25<01:22, 82.61s/it]


Epoch 009 train_loss: 1.2982     val_loss 1.3158 train_perplexirty 3.6638 val_perplexirty 3.7291


epoch: 100%|██████████| 10/10 [13:48<00:00, 82.83s/it]


Epoch 010 train_loss: 1.2819     val_loss 1.3065 train_perplexirty 3.6046 val_perplexirty 3.6946
Best val perplexirty: 3.694581


In [ ]:
# Тестирование генерации текста
generate_sequence(
    model, 
    char2ind, 
    ind2char, 
    starting_seq='источник связан с '
)

'<bos>источник связан с помощью статьи по проекту<unk><eos>'

### Пословная токенизация

In [ ]:
# Очистка памяти перед переходом к словесной модели
torch.cuda.empty_cache()
gc.collect()

485

In [ ]:
# Токенизация на уровне слов
words = Counter()

for sentence in tqdm(sentences):
    for word in nltk.word_tokenize(sentence):
            words[word] += 1
            
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 40000

for elem in words.most_common(vocab_size):
    vocab.add(elem[0])
    
print("Всего слов в словаре:", len(vocab))

100%|██████████| 120873/120873 [00:29<00:00, 4096.45it/s]


Всего слов в словаре: 40004


In [25]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
# Переопределение функции паддинга для словесных данных
def collate_fn_with_padding(
    input_batch: List[List[int]], 
    pad_id: int = word2ind['<pad>']
) -> Dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для преобразования батча данных: добавляет паддинг и 
        создает тензоры ввода и целевых значений.

    Args:
    ---------------
        input_batch: Пакет токенизированных предложений
        pad_id: Идентификатор токена для заполнения (паддинга)

    Returns:
    ---------------
        Dict[str, torch.Tensor]: Словарь с тензорами входных и целевых id
    """
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch

In [ ]:
# Создание датасетов и загрузчиков для словесной модели
train_sentences, eval_sentences = train_test_split(
    sentences, test_size=0.2
)

train_dataset = WordDataset(train_sentences)
eval_dataset  = WordDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=64
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=64
)

In [ ]:
# Создание и обучение однослойной LSTM для словесной модели
model = LanguageModel(
    hidden_dim=256, 
    vocab_size=len(vocab), 
    num_layers=1
).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

LanguageModel(
  (embedding): Embedding(40004, 256)
  (lstm_layers): ModuleList(
    (0): LSTM(256, 256, batch_first=True)
  )
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=40004, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)
Number of model parameters: 21,114,180


In [ ]:
best_model, losses = train(
    train_dataloader, 
    eval_dataloader, 
    model, 
    10, 
    ignore_index=word2ind["<pad>"]
)

epoch:  10%|█         | 1/10 [01:47<16:04, 107.18s/it]


Epoch 001 train_loss: 6.3350     val_loss 5.8254 train_perplexirty 832.8000 val_perplexirty 343.8051


epoch:  20%|██        | 2/10 [03:34<14:17, 107.14s/it]


Epoch 002 train_loss: 5.5044     val_loss 5.4580 train_perplexirty 251.8205 val_perplexirty 238.4559


epoch:  30%|███       | 3/10 [05:21<12:31, 107.35s/it]

...


Epoch 008 train_loss: 3.8872     val_loss 5.3552 train_perplexirty 49.5441 val_perplexirty 217.7147


epoch:  90%|█████████ | 9/10 [16:09<01:47, 107.99s/it]


Epoch 009 train_loss: 3.7329     val_loss 5.4208 train_perplexirty 42.4665 val_perplexirty 232.9360


epoch: 100%|██████████| 10/10 [17:57<00:00, 107.77s/it]


Epoch 010 train_loss: 3.5940     val_loss 5.4889 train_perplexirty 36.9549 val_perplexirty 249.8979
Best val perplexirty: 187.842122


In [ ]:
# Тестирование генерации текста
generate_sequence(
    model, 
    word2ind, 
    ind2word, 
    starting_seq=nltk.word_tokenize('история россии')
)


'<bos> история россии – это наука о росте народонаселения . <eos>'

### Добавим несколько слоев LSTM

In [ ]:
# Очистка памяти для следующей модели
torch.cuda.empty_cache()
gc.collect()

1995

In [ ]:
# Создание и обучение многослойной LSTM для словесной модели
model = LanguageModel(
    hidden_dim=256, 
    vocab_size=len(vocab), 
    num_layers=3
).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

LanguageModel(
  (embedding): Embedding(40004, 256)
  (lstm_layers): ModuleList(
    (0-2): 3 x LSTM(256, 256, batch_first=True)
  )
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=40004, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)
Number of model parameters: 22,166,852


In [ ]:
best_model, losses = train(
    train_dataloader, 
    eval_dataloader, 
    model, 
    10, 
    ignore_index=word2ind["<pad>"]
)

epoch:  10%|█         | 1/10 [02:05<18:45, 125.02s/it]


Epoch 001 train_loss: 6.3070     val_loss 5.8164 train_perplexirty 758.6220 val_perplexirty 340.7918


epoch:  20%|██        | 2/10 [04:09<16:39, 124.92s/it]


Epoch 002 train_loss: 5.4720     val_loss 5.4439 train_perplexirty 244.0570 val_perplexirty 235.2224


epoch:  30%|███       | 3/10 [06:14<14:34, 124.92s/it]

...


Epoch 008 train_loss: 3.8201     val_loss 5.3841 train_perplexirty 46.3572 val_perplexirty 224.0727


epoch:  90%|█████████ | 9/10 [18:45<02:05, 125.10s/it]


Epoch 009 train_loss: 3.6598     val_loss 5.4538 train_perplexirty 39.4794 val_perplexirty 240.7387


epoch: 100%|██████████| 10/10 [20:50<00:00, 125.01s/it]


Epoch 010 train_loss: 3.5142     val_loss 5.5317 train_perplexirty 34.1306 val_perplexirty 260.7878
Best val perplexirty: 191.083743


In [ ]:
# Тестирование генерации текста
generate_sequence(
    model, 
    word2ind, 
    ind2word,
    starting_seq=nltk.word_tokenize('история россии определяется')
)

'<bos> история россии определяется населением в россии . <eos>'